In [1]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
BOS, EOS = "<s>", "</s>"

# fmt: off
DEFAULT_SYSTEM_PROMPT = """You are a knowledgeable agriculture and farming expert and you don't know about anything else other than agricultural science and farming
. Your purpose is to provide helpful and user-friendly information on agriculture-related topics. Please respond only to questions directly related to agriculture and farming. If the question is not related, simply reply with "It is not related to agriculture."
Please only reply to Agriculture and Farming and agriculture science related questions.
Please do not reply to any other questions asked by the user.
don't try to explain the user's question if the question is not related to agriculture or farming.
Please give your answers in a user-friendly way and easy to understand.
Please do not give any answers that are not related to agriculture and farming.
Please skip the user's question if the question is not about agriculture and farming and agriculture science.

Examples:
Question: what is love?
Answer: As it is not related to agriculture I can't answer that question Please ask me ssomething related to Agriculture or farming.
Question: What are the best practices for growing tomatoes?
Answer: Tomatoes need full sun and well-drained soil. Water them regularly, but be careful not to overwater them. Fertilize them every few weeks with a balanced fertilizer. Harvest tomatoes when they are ripe and red.
Question: What are the symptoms of corn smut?
Answer: Corn smut is a fungal disease that can affect all parts of the corn plant. Symptoms include black, powdery galls on the leaves, stalks, and ears of corn. Corn smut is not harmful to humans, but it can reduce crop yields.
Question: How do I get rid of weeds in my garden?
Answer: There are many ways to get rid of weeds in your garden. You can hand-pull them, hoe them, or use herbicides. If you use herbicides, be sure to follow the directions on the label carefully."""
# fmt: on

In [2]:
from typing import Literal, TypedDict, List

Role = Literal["system", "user", "assistant"]

class Message(TypedDict):
    role: Role
    content: str
    
Dialog = List[Message]

In [3]:
dialog: Dialog = [
    {"role": "system","content": DEFAULT_SYSTEM_PROMPT,},
    {"role": "user", "content": "How to make someone fall for me?"},
]
if dialog[0]["role"] == "system":
    dialog = [
        {
            "role": dialog[1]["role"],
            "content": B_SYS + dialog[0]["content"] + E_SYS + dialog[1]["content"],
        }
    ] + dialog[2:]


In [5]:
print('loading model')
path = "/mnt/Ddrive/Llama-2-7B-Chat-AWQ"
from vllm import AsyncEngineArgs, AsyncLLMEngine, SamplingParams, RequestOutput
engine_args = AsyncEngineArgs(model=path, max_model_len=1024, quantization="awq", block_size=8)
engine = AsyncLLMEngine.from_engine_args(engine_args)
sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=256)

loading model
WARNING 01-23 22:21:27 config.py:175] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 01-23 22:21:27 llm_engine.py:70] Initializing an LLM engine with config: model='/mnt/Ddrive/Llama-2-7B-Chat-AWQ', tokenizer='/mnt/Ddrive/Llama-2-7B-Chat-AWQ', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=awq, enforce_eager=False, seed=0)
INFO 01-23 22:21:30 llm_engine.py:275] # GPU blocks: 469, # CPU blocks: 1024
INFO 01-23 22:21:31 model_runner.py:501] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 01-23 22:21:31 model_runner.py:505] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consid

In [6]:
def get_tokens(dialog: Dialog) -> str:
    dialog_tokens = sum(
        [
            [f"{BOS}{B_INST} {(prompt['content']).strip()} {E_INST} {(answer['content']).strip()}{EOS}"]
            for prompt, answer in zip(dialog[::2], dialog[1::2],)
        ],
        [],
    )
    dialog_tokens += f"{BOS}{B_INST} {(dialog[-1]['content']).strip()} {E_INST}{EOS}",
    return ''.join(dialog_tokens)

In [7]:
get_tokens(dialog)

'<s>[INST] <<SYS>>\nYou are a knowledgeable agriculture and farming expert and you don\'t know about anything else other than agricultural science and farming\n. Your purpose is to provide helpful and user-friendly information on agriculture-related topics. Please respond only to questions directly related to agriculture and farming. If the question is not related, simply reply with "It is not related to agriculture."\nPlease only reply to Agriculture and Farming and agriculture science related questions.\nPlease do not reply to any other questions asked by the user.\ndon\'t try to explain the user\'s question if the question is not related to agriculture or farming.\nPlease give your answers in a user-friendly way and easy to understand.\nPlease do not give any answers that are not related to agriculture and farming.\nPlease skip the user\'s question if the question is not about agriculture and farming and agriculture science.\n\nExamples:\nQuestion: what is love?\nAnswer: As it is no

In [10]:
generated = ""
async for out in engine.generate(get_tokens(dialog), sampling_params=sampling_params, request_id="inference"):
    new_generated = out.outputs[0].text.replace(generated, "")
    print(new_generated, end="")
    generated = out.outputs[0].text
dialog.append({"role": "assistant", "content": generated})

INFO 01-23 22:22:56 async_llm_engine.py:383] Received request inference: prompt: '<s>[INST] <<SYS>>\nYou are a knowledgeable agriculture and farming expert and you don\'t know about anything else other than agricultural science and farming\n. Your purpose is to provide helpful and user-friendly information on agriculture-related topics. Please respond only to questions directly related to agriculture and farming. If the question is not related, simply reply with "It is not related to agriculture."\nPlease only reply to Agriculture and Farming and agriculture science related questions.\nPlease do not reply to any other questions asked by the user.\ndon\'t try to explain the user\'s question if the question is not related to agriculture or farming.\nPlease give your answers in a user-friendly way and easy to understand.\nPlease do not give any answers that are not related to agriculture and farming.\nPlease skip the user\'s question if the question is not about agriculture and farming an

In [9]:
i = input().strip()
dialog.append({"role": "user", "content": i})